In [1]:
import numpy as np
import pandas as pd
from itertools import product
from datetime import datetime
import plotly.express as px
from dlt_utils import DLTReader
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pyspark.sql.functions import current_date, add_months, col
from dateutil.relativedelta import relativedelta

In [2]:
## Loading Customer Master

dlt_reader = DLTReader(
    catalog="provisioned-tableau-data", 
    schema="tableau_delta_tables"
)

customer_data = dlt_reader.read_table("customer_master")
customer_data = customer_data.filter(col("dealer_group") == "Z001") ## Getting Replacement dealers only
customer_master = customer_data.toPandas()

## Filtering out only Z001 dealers
dealer_list = customer_master['dealer_code'].to_list()

⚡ Initializing Databricks Spark session...
✅ Session initialized successfully.
📁 Setting catalog to: `provisioned-tableau-data`
📄 Setting schema to: `tableau_delta_tables`
📖 Reading table: `customer_master`...
✅ Successfully read data from `customer_master`.


In [3]:
len(dealer_list)

9259

In [4]:
## Laoding Sales Data

sales = dlt_reader.read_table("sales_data")
sales = dlt_reader.read_table("sales_data").filter(
    (col("dealer_code").isin([str(dealer) for dealer in dealer_list])) &
    (col("indicator_cancel").isNull()) &
    (~col("invoice_type").isin(['S1', 'S2', 'S3', 'S4']))
)

sales_data = sales.toPandas()

📖 Reading table: `sales_data`...
✅ Successfully read data from `sales_data`.
📖 Reading table: `sales_data`...
✅ Successfully read data from `sales_data`.


In [5]:
sales_data['dealer_code']= sales_data['dealer_code'].astype(str)
sales_data['dealer_code'].nunique()

9242

In [6]:
## Filtering data, removing current month data
sales_data['invoice_date'] = pd.to_datetime(sales_data['invoice_date'])

reference_date = (sales_data['invoice_date'].max().replace(day=1) - pd.Timedelta(days=1))

##Dataoffset
one_year_ago = reference_date - pd.DateOffset(years=1)

sales_data = sales_data[
    (sales_data['invoice_date'] <= reference_date)
    # (sales_data['invoice_date'] > one_year_ago)
]

In [7]:
## Getting last invoice_date and last_billed_days
sales_data['invoice_date'] = pd.to_datetime(sales_data['invoice_date'])
 
last_billed = sales_data.groupby('dealer_code')['invoice_date'].max().reset_index(name='last_invoice_date')
last_billed['last_billed_days'] = (reference_date - last_billed['last_invoice_date']).dt.days

## Getting churn labels
last_billed['churn_status'] = np.where(
    last_billed['last_billed_days'] > 90, 
    'Churned', 
    'Active'
)

In [8]:
sales_data['period'] = sales_data['period'].astype(str)

all_periods = sales_data['period'].unique()
monthly_sales = sales_data.groupby(['dealer_code','period'])['ndp_value'].sum().reset_index()
all_dealers = sales_data['dealer_code'].unique()

full_index = pd.DataFrame(product(all_dealers, all_periods), columns=['dealer_code', 'period'])
monthly_sales = sales_data.groupby(['dealer_code', 'period'])['ndp_value'].sum().reset_index()

full_sales = pd.merge(full_index, monthly_sales, on=['dealer_code', 'period'], how='left')

full_sales['no_sales_flag'] = full_sales['ndp_value'].isna().astype(int)
full_sales['ndp_value'] = full_sales['ndp_value'].fillna(0)

In [9]:
def get_max_streak_info(group):
    max_streak = 0
    current_streak = 0
    streak_start = None
    max_streak_start = None

    for idx, row in group.iterrows():
        if row['no_sales_flag'] == 1:
            if current_streak == 0:
                streak_start = row['period']
            current_streak += 1
            if current_streak > max_streak:
                max_streak = current_streak
                max_streak_start = streak_start
        else:
            current_streak = 0
            streak_start = None

    return pd.Series({
        'max_no_sales_streak': max_streak,
        'max_streak_start_period': max_streak_start
    })

streak_info_df = full_sales.groupby('dealer_code').apply(get_max_streak_info).reset_index()


/var/folders/f7/p98720yn5kn5tvm33x4lx_89rys50w/T/ipykernel_11202/44624538.py:24: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  streak_info_df = full_sales.groupby('dealer_code').apply(get_max_streak_info).reset_index()


In [10]:
last_billed = pd.merge(last_billed, streak_info_df,on="dealer_code", how='left')

In [11]:
last_billed['max_streak_start_period'] = pd.to_datetime(last_billed['max_streak_start_period'])

## Conditions for Active as well as Churn Dealers
condition1 = (
    (last_billed['churn_status'] == "Churned") &
    (last_billed['max_no_sales_streak'] > 4) &
    (last_billed['max_streak_start_period'] > pd.to_datetime('2022-11'))
)

condition2 = (
    (last_billed['churn_status'] == "Active") &
    (
        (last_billed['max_streak_start_period'] <= one_year_ago) |
        (last_billed['max_streak_start_period'].isna() )
    )
)

# First apply condition1
last_billed['threshold_end_date'] = np.where(
    condition1,
    last_billed['max_streak_start_period'],
    pd.NaT  
)

# Then apply condition2 only where condition1 is False and condition2 is True
mask = (~condition1) & (condition2)
last_billed.loc[mask, 'threshold_end_date'] = reference_date

# Finally, for remaining nulls in threshold_end_date, fill with max_streak_start_period
last_billed['threshold_end_date'] = last_billed['threshold_end_date'].fillna(
    last_billed['max_streak_start_period']
)

last_billed['threshold_end_date'] = pd.to_datetime(last_billed['threshold_end_date'])

## Removing dealer whose complete 1 year journey we don't have
last_billed = last_billed[last_billed['threshold_end_date']>pd.to_datetime('2022-12')]

last_billed['threshold_start_date'] = last_billed['threshold_end_date'] - pd.DateOffset(months=12)

In [12]:
last_billed['threshold_end_date'] = (last_billed['threshold_end_date'].dt.to_period('M')).astype(str)
last_billed['threshold_start_date'] = (last_billed['threshold_start_date'].dt.to_period('M')).astype(str)
sales_journey = pd.merge(sales_data, last_billed,on='dealer_code',how='left')

In [13]:
## Performing Date Offset
sales_journey['invoice_date'] = pd.to_datetime(sales_journey['invoice_date'], errors='coerce')
sales_journey = sales_journey[
    (sales_journey['invoice_date'] >= sales_journey['threshold_start_date']) &
    (sales_journey['invoice_date'] <= sales_journey['threshold_end_date'])
].copy()


In [14]:
df = pd.merge(
    sales_journey, 
    customer_master, 
    on='dealer_code', 
    how='left'
)

In [15]:
## Reading Data
dlt_reader = DLTReader(
    catalog="provisioned-tableau-data", 
    schema="tableau_delta_tables"
)

## Claims Data
claims = dlt_reader.read_table("claims_data")
claims = claims.filter(claims.DealerCode.isin([str(dealer) for dealer in dealer_list]))
claims_data = claims.toPandas()

## Territory Master
territory_master = dlt_reader.read_table("territory_master")
territory_master = territory_master.toPandas()

## Visits Data
visits = dlt_reader.read_table("visits_flat")
visits = visits.filter(visits.dealerCode.isin([str(dealer) for dealer in dealer_list]))
visits_data = visits.toPandas()

## SAS Monthly
sas_monthly = dlt_reader.read_table("monthly_club_mapping")
sas_monthly = sas_monthly.filter(sas_monthly.dealer_code.isin([str(dealer) for dealer in dealer_list]))
sas_monthly_data = sas_monthly.toPandas()

## Credit Note
credit_note = dlt_reader.read_table("credit_note")
credit_note = credit_note.filter(credit_note.dealer_code.isin([str(dealer) for dealer in dealer_list]))
credit_note_df = credit_note.toPandas()

## Reading Product Master 
pm = dlt_reader.read_table("product_master")
product_data = pm.toPandas()

# ## Reading Dealer Exposure Data
dlt_reader = DLTReader(
    catalog="provisioned-tableau-data", 
    schema="sap_data"
)

outstanding = dlt_reader.read_table("customer_financial")
# F.col("dealer_code").contains(dealer_list)
outstanding = outstanding.filter(outstanding.dealer_code.isin([str(dealer) for dealer in dealer_list]))
outstanding_df = outstanding.toPandas()

### Reading Dealer orders Data
dlt_reader = DLTReader(
    catalog="provisioned-tableau-data", 
    schema="jkc"
)

orders = dlt_reader.read_table("orders")
orders = orders.filter(orders.Dealercode.isin([str(dealer) for dealer in dealer_list]))
orders_df = orders.toPandas()



⚡ Initializing Databricks Spark session...
✅ Session initialized successfully.
📁 Setting catalog to: `provisioned-tableau-data`
📄 Setting schema to: `tableau_delta_tables`
📖 Reading table: `claims_data`...
✅ Successfully read data from `claims_data`.
📖 Reading table: `territory_master`...
✅ Successfully read data from `territory_master`.
📖 Reading table: `visits_flat`...
✅ Successfully read data from `visits_flat`.
📖 Reading table: `monthly_club_mapping`...
✅ Successfully read data from `monthly_club_mapping`.
📖 Reading table: `credit_note`...
✅ Successfully read data from `credit_note`.
📖 Reading table: `product_master`...
✅ Successfully read data from `product_master`.
⚡ Initializing Databricks Spark session...
✅ Session initialized successfully.
📁 Setting catalog to: `provisioned-tableau-data`
📄 Setting schema to: `sap_data`
📖 Reading table: `customer_financial`...
✅ Successfully read data from `customer_financial`.
⚡ Initializing Databricks Spark session...
✅ Session initialized su

In [16]:
## Dealership Age 
customer_master['dealership_age'] = (datetime.now() - pd.to_datetime(customer_master['creation_date'])).dt.days //365

## Monthly Sales Data

In [17]:
## Calculating Total Sales trajectory over last 1 year
monthly_sales = sales_journey.groupby(['dealer_code','period']).agg(
    total_sales=('ndp_value', 'sum'),
    total_invoices=('invoice_number', 'nunique'),
    total_units=('volume', 'sum'),
).reset_index()

## Calculating Average Sales trajectory over last 1 year
monthly_sales['avg_invoices'] = monthly_sales['total_invoices']/ monthly_sales['period'].nunique()
monthly_sales['avg_sales'] = monthly_sales['total_sales']/ monthly_sales['period'].nunique()
monthly_sales['avg_units_purchased'] = monthly_sales['total_units']/ monthly_sales['period'].nunique()

## Calculating Trends
# monthly_sales[['mom_change_avg_sales','mom_change_avg_invoices','mom_change_avg_units_purchased']] = monthly_sales.groupby('dealer_code')[['avg_sales','avg_invoices','avg_units_purchased']].pct_change()

## Getting territory_code and dealer_club to getting similar terr,club trends
monthly_sales = pd.merge(monthly_sales, customer_master[['dealer_code', 'territory_code','dealer_club_category']], on='dealer_code', how='left')

## Territory Level Sales Trends
# Calculating average sales, invoices, and units for each territory
terrwise_monthly_sales = monthly_sales.groupby(['territory_code', 'period'])[['total_sales','total_invoices','total_units']].mean().reset_index().rename(
    columns={
        'total_sales': 'avg_sales_of_dealers_in_same_territory',
        'total_invoices': 'avg_orders_of_dealers_in_same_territory',
        'total_units': 'avg_units_of_dealers_in_same_territory'
    })

## Dealer Club Level Sales Trends
# Calculating average sales, invoices, and units for each dealer club category
dealerclubwise_monthly_sales = monthly_sales.groupby(['dealer_club_category', 'period'])[['total_sales','total_invoices','total_units']].mean().reset_index().rename(
    columns={
        'total_sales': 'avg_sales_of_dealers_in_same_dealer_club_category',
        'total_invoices': 'avg_orders_of_dealers_in_dealer_club_category',
        'total_units': 'avg_units_of_dealers_in_dealer_club_category'
    })

##Calculating MOM change in sales dealerclub-wise as well as territory-wise
## Calculating %age change in sales dealer-club-wise 
# dealerclubwise_monthly_sales[['pct_change_club_sales','pct_change_club_orders','pct_change_club_salesunits']] = dealerclubwise_monthly_sales.groupby('dealer_club_category')[['avg_sales_of_dealers_in_same_dealer_club_category',
#        'avg_orders_of_dealers_in_dealer_club_category',
#        'avg_units_of_dealers_in_dealer_club_category']].pct_change()

## Calculating %age change in sales territory-wise
# terrwise_monthly_sales[['pct_change_terr_sales','pct_change_terr_orders','pct_change_terr_salesunits']] = terrwise_monthly_sales.groupby('territory_code')[['avg_sales_of_dealers_in_same_territory',
#                                                                                                                                                             'avg_orders_of_dealers_in_same_territory',
#                                                                                                                                                             'avg_units_of_dealers_in_same_territory']].pct_change()

## merging all data
monthly_sales = pd.merge(monthly_sales, terrwise_monthly_sales, on=['territory_code','period'],how='left') \
.merge(dealerclubwise_monthly_sales, on=['dealer_club_category','period'], how='left')

In [18]:
monthly_sales.head()

,dealer_code,period,total_sales,total_invoices,total_units,avg_invoices,avg_sales,avg_units_purchased,territory_code,dealer_club_category,avg_sales_of_dealers_in_same_territory,avg_orders_of_dealers_in_same_territory,avg_units_of_dealers_in_same_territory,avg_sales_of_dealers_in_same_dealer_club_category,avg_orders_of_dealers_in_dealer_club_category,avg_units_of_dealers_in_dealer_club_category
0,1100001,2024-07,414471.0,35,166.0,0.813953,9638.860465,3.860465,321201,BLUE CLUB,1.292423e+06,54.357143,279.071429,360720.013558,22.998210,102.717917
1,1100001,2024-08,704437.0,52,210.0,1.209302,16382.255814,4.883721,321201,BLUE CLUB,1.408279e+06,54.428571,293.285714,359253.359993,22.465566,103.318946
2,1100001,2024-09,471263.0,30,110.0,0.697674,10959.604651,2.558140,321201,BLUE CLUB,1.179779e+06,47.071429,260.428571,372118.825476,22.855037,108.017424
3,1100001,2024-10,259637.0,22,80.0,0.511628,6038.069767,1.860465,321201,BLUE CLUB,1.309881e+06,44.928571,276.000000,377761.737210,22.646261,104.266650
4,1100001,2024-11,196954.0,28,70.0,0.651163,4580.325581,1.627907,321201,BLUE CLUB,1.476041e+06,53.266667,307.066667,366647.755778,21.982753,100.525699


In [19]:
## Calculate the day between consecutive puchases
sales_journey['invoice_date'] = pd.to_datetime(sales_journey['invoice_date'])
sales_journey = sales_journey.sort_values(['dealer_code', 'invoice_date'])
sales_journey['days_between_purchases'] = sales_journey.groupby(['dealer_code'])['invoice_date'].diff().dt.days
days_between_purchase = sales_journey.groupby('dealer_code')['days_between_purchases'].mean().reset_index(name="avg_days_between_purchase")
sales_journey = pd.merge(sales_journey,days_between_purchase,on='dealer_code', how='left')

In [20]:
monthly_sales.head()

,dealer_code,period,total_sales,total_invoices,total_units,avg_invoices,avg_sales,avg_units_purchased,territory_code,dealer_club_category,avg_sales_of_dealers_in_same_territory,avg_orders_of_dealers_in_same_territory,avg_units_of_dealers_in_same_territory,avg_sales_of_dealers_in_same_dealer_club_category,avg_orders_of_dealers_in_dealer_club_category,avg_units_of_dealers_in_dealer_club_category
0,1100001,2024-07,414471.0,35,166.0,0.813953,9638.860465,3.860465,321201,BLUE CLUB,1.292423e+06,54.357143,279.071429,360720.013558,22.998210,102.717917
1,1100001,2024-08,704437.0,52,210.0,1.209302,16382.255814,4.883721,321201,BLUE CLUB,1.408279e+06,54.428571,293.285714,359253.359993,22.465566,103.318946
2,1100001,2024-09,471263.0,30,110.0,0.697674,10959.604651,2.558140,321201,BLUE CLUB,1.179779e+06,47.071429,260.428571,372118.825476,22.855037,108.017424
3,1100001,2024-10,259637.0,22,80.0,0.511628,6038.069767,1.860465,321201,BLUE CLUB,1.309881e+06,44.928571,276.000000,377761.737210,22.646261,104.266650
4,1100001,2024-11,196954.0,28,70.0,0.651163,4580.325581,1.627907,321201,BLUE CLUB,1.476041e+06,53.266667,307.066667,366647.755778,21.982753,100.525699


## Monthly SAS Data

In [21]:
sas_monthly_data['period'].min(), sas_monthly_data['period'].max()

('2022-01', '2025-09')

In [22]:
sas_monthly_data['dealer_code'] = sas_monthly_data['dealer_code'].astype(str)

sas_monthly_data['upd_period'] = (pd.to_datetime(sas_monthly_data['period'], format='%Y-%m') - pd.DateOffset(months=1)).dt.strftime('%Y-%m')

# Filtering SAS data for given reference date
sas_monthly_data = sas_monthly_data[sas_monthly_data['upd_period'] <= reference_date.strftime('%Y-%m')]

In [23]:
sas_monthly_data['period'] = sas_monthly_data['upd_period'].astype(str)

In [24]:
monthly_data = pd.merge(monthly_sales, sas_monthly_data[['dealer_code','sas_amount','period']], on=['dealer_code', 'period'], how='left')

In [25]:
monthly_data['rotation'] = monthly_data['total_sales']/monthly_data['sas_amount']

In [26]:
terrwise_sas = monthly_data.groupby(['territory_code', 'period'])['rotation'].mean().reset_index(name='terrwise_rotation')
dealerclub_wise_sas = monthly_data.groupby(['dealer_club_category', 'period'])['rotation'].mean().reset_index(name='dealerclub_wise_rotation')

In [27]:
monthly_data = pd.merge(monthly_data,terrwise_sas, on=['territory_code','period'],how='left') \
.merge(dealerclub_wise_sas, on = ['dealer_club_category','period'],how = 'left')

In [28]:
sas_monthly_data['prev_dealer_club'] = sas_monthly_data.groupby('dealer_code')['dealer_club'].shift(1)

club_hierarchy = {
    'Non Starter': 0,
    'Starter': 1,
    'Blue Club': 2,
    'Gold Plus Club': 3,
    'Platinum Club': 4,
    'Diamond Club':5,
    'Acer Club': 6,
    "Chairman's Club":7,
    "Chairman's Advisory Club": 8
}

sas_monthly_data['curr_level'] = sas_monthly_data['dealer_club'].map(club_hierarchy)
sas_monthly_data['prev_level'] = sas_monthly_data['prev_dealer_club'].map(club_hierarchy)

def classify_movement(row):
    if pd.isna(row['prev_level']):
        return np.nan
    elif row['curr_level'] > row['prev_level']:
        return 'Promoted'
    elif row['curr_level'] < row['prev_level']:
        return 'Demoted'
    else:
        return 'No Change'

sas_monthly_data['club_movement'] = sas_monthly_data.apply(classify_movement, axis=1)

movement_counts = (
    sas_monthly_data
    .groupby(['dealer_code', 'club_movement'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)

In [29]:
##TODO: TO BE ADDED: 
# - days between purchases
# - movement_counts

### Monthly Collection/ Outstanding/ Exposure

In [30]:
outstanding_df['period'] = pd.to_datetime(outstanding_df['period'], format='%Y%m').dt.strftime('%Y-%m')


In [31]:
monthly_data = pd.merge(monthly_data, outstanding_df, on=['dealer_code', 'period'], how='left')

### Monthly Credit Note Data

In [32]:
credit_note_df.groupby(['dealer_code', 'period'])['note_value'].nunique()

dealer_code  period 
1100001      2022-01     7
             2022-02     9
             2022-03    13
             2022-04     1
             2022-05     8
                        ..
1123158      2025-08     3
1123159      2025-08     1
1123165      2025-08     3
1123167      2025-08     1
1123180      2025-08     1
Name: note_value, Length: 227871, dtype: int64

In [33]:
monthly_credit_note = credit_note_df.groupby(['dealer_code', 'period']).agg(
    total_credit_note_value=('note_value', 'sum'),
    avg_credit_note_value=('note_value', 'mean')        
).reset_index()

In [34]:
monthly_data = pd.merge(monthly_data,monthly_credit_note, on=['dealer_code', 'period'], how='left')

## Monthly Online vs Offline Orders

In [35]:
orders_df.head()

,ALL,BillAmount,Dealercode,IMEInumber,InvDate,Meins,NdpVal,Ntgew,OrderNo,SoNo,...,product_invoiceNumber,product_materialNo,product_prc,product_product,product_productStatus,product_quantity,product_quantityCancelled,product_receivedQuantity,product_sgst,product_taxValue
0,None,16576.0,1117600,29b3256ae9014c09,2024-05-24 05:57:27.750,None,None,None,033739797600,None,...,None,None,NaN,None,None,NaN,NaN,NaN,NaN,NaN
1,None,36275.2,1106865,1716528050568,2024-05-24 05:57:33.151,None,None,None,071440686865,None,...,None,None,NaN,None,None,NaN,NaN,NaN,NaN,NaN
2,None,77644.8,1120101,b247741235dc1abf,2024-05-24 05:57:42.085,None,None,None,048374930101,None,...,None,None,NaN,None,None,NaN,NaN,NaN,NaN,NaN
3,None,18329.6,1101145,1716529465823,2024-05-24 05:57:47.947,None,None,None,099906721145,None,...,None,None,NaN,None,None,NaN,NaN,NaN,NaN,NaN
4,None,12134.4,1117598,2b594eae8c7e279c,2024-05-24 05:57:57.373,None,None,None,080089827598,None,...,None,None,NaN,None,None,NaN,NaN,NaN,NaN,NaN


In [36]:
sales_data['invoice_number'] = sales_data['invoice_number'].apply(lambda x: str(x).split('.')[0] if pd.notna(x) else x)
orders_df.rename(columns={'products_invoiceNumber': 'invoice_number', 'Dealercode': 'dealer_code'}, inplace=True)

t1 = pd.merge(sales_data[['dealer_code','invoice_number','invoice_date','volume','indicator_cancel', 'ndp_value','invoice_type','period']], orders_df[['dealer_code', 'invoice_number']],
          on=['dealer_code', 'invoice_number'],
          how='left',
          indicator=True
          )

## filtering invoices 
t1['order_type'] = np.where(t1['_merge'] == 'both', 'online', 'offline')

## Merging Data
order_types = t1.drop_duplicates(subset='invoice_number').groupby(['dealer_code','period'])['order_type'].value_counts().unstack(fill_value=0).reset_index()
order_types['total_orders'] = order_types[['online', 'offline']].sum(axis=1)

In [37]:
monthly_data = pd.merge(monthly_data,order_types, on=['dealer_code', 'period'], how='left')

## Monthly Claims Data

In [38]:
claims_data['period'] = pd.to_datetime(claims_data['ClaimDate']).dt.to_period('M').astype(str)

# List of statuses that represent a settled claim
settled_statuses = [
    'Inspection Accepted',
    'Inspection Rejected',
    'AI : Inspection Rejected'
]

# Create a new column flagging settled vs active
claims_data['claim_state'] = claims_data['ClaimStatus'].apply(
    lambda x: 'Settled' if x in settled_statuses else 'Active'
)

total_claims = (
    claims_data
    .groupby(['DealerCode','period'])['ClaimNo']
    .nunique()
    .reset_index(name='unique_claims')
)

settled_claims = claims_data[claims_data['claim_state'] == 'Settled'].drop_duplicates("ClaimNo").groupby(['DealerCode','period'])['ClaimNo'].count().reset_index(name='settled_claims')

claim_count = pd.merge(total_claims, settled_claims, on = ['DealerCode','period'],how='left')
claim_count["active_claims"] = claim_count['unique_claims'] - claim_count['settled_claims']

In [39]:
claim_count.rename(columns={'DealerCode': 'dealer_code'}, inplace=True)

In [40]:
monthly_data = pd.merge(monthly_data,claim_count, on=['dealer_code', 'period'], how='left')

## Monthly Visits Data

In [41]:
visits_data.head()

,SAP_ID,__v,_id,completed,createdAt,dealerCode,dealerFeedback,deleted,endTime,happinessScore,...,visitAction_actionTaken,visitAction_completed,visitAction_feedback,visitAction_imageCapturedUrl,visitAction_locationString,visitAction_type,visitAction_feedback_photos,visitAction_feedback_subject,visitAction_feedback_text,visitAction_feedback_type
0,SO361301,0,6447a72b263e8530b59467c7,True,2023-04-25 10:10:51.885,1119424,None,False,2023-04-25 15:45:52.588,None,...,gsb,True,None,https://jkmsfa.s3.ap-south-1.amazonaws.com/pub...,"Sikar, Industrial Area, April 25, 2023 3:40:58 PM",image-capture,[],None,None,None
1,SO361301,0,6447a72b263e8530b59467c7,True,2023-04-25 10:10:51.885,1119424,None,False,2023-04-25 15:45:52.588,None,...,discuss with the dealer for the sas increase a...,True,None,None,None,performance,[],None,None,None
2,SO361301,0,6447a72b263e8530b59467c7,True,2023-04-25 10:10:51.885,1119424,None,False,2023-04-25 15:45:52.588,None,...,ask for the sas and payment as per due date,True,None,None,None,settle-collection,[],None,None,None
3,SO361301,0,6447a72b263e8530b59467c7,True,2023-04-25 10:10:51.885,1119424,None,False,2023-04-25 15:45:52.588,None,...,ask dealer for get bill the 155/80/12 Bill and...,True,None,None,None,stock-record,[],None,None,None
4,SO361301,0,6447a72b263e8530b59467c7,True,2023-04-25 10:10:51.885,1119424,None,False,2023-04-25 15:45:52.588,None,...,dealer place the order and get bill the tyre a...,True,None,None,None,feedback,[],None,None,None


In [42]:
visits_data = visits_data[visits_data['completed'] == True]
visits_data['period'] = pd.to_datetime(visits_data['updatedAt']).dt.to_period('M').astype(str)

visit_count = visits_data.groupby(['dealerCode','period'])['_id'].nunique().reset_index(name='visit_count')


In [43]:
visit_count.rename(columns={'dealerCode': 'dealer_code'}, inplace=True)
monthly_data = pd.merge(monthly_data,visit_count, on=['dealer_code', 'period'], how='left')

## Counter Share Data

In [44]:

cs_25 = pd.read_excel("/Users/lakshita.kain/Downloads/Personal Data/GST DATA/Market Mapping  New GST.xlsx")
pin_map = pd.read_excel("/Users/lakshita.kain/Downloads/Personal Data/GST DATA/Market Mapping  New GST.xlsx",sheet_name="PIN")

cs_25.columns = cs_25.columns.str.lower()
pin_map.columns = pin_map.columns.str.lower()

cs_25.rename(columns=
    {
        'pin': 'pincode'
    },
    inplace=True
)

cs_25 = pd.merge(
    cs_25,
    pin_map[['pincode', 'state', 'district','terr. code']],
    on='pincode',
    how='left'
)

companies = ['jktil', 'apollo', 'mrf', 'ceat', 'oth.']
cs_25[companies] = cs_25[companies].apply(pd.to_numeric, errors='coerce')

cs_25['terr. code'].replace({'RANCHI-III':'171501', 'RANCHI-IV':'171502','DHANBAD': '172201', 'DUMKA':'172301' ,'JAMSHEDPUR':'171101' }, inplace=True)

# Group by territory and sum company values
territory_sum = cs_25.groupby('terr. code')[companies].sum().reset_index()

# Calculate total across companies per territory
territory_sum['total'] = territory_sum[companies].sum(axis=1)

# Calculate percentage share for each company
for company in companies:
    territory_sum[f'{company}_share'] = (territory_sum[company] / territory_sum['total']) * 100

territory_sum.rename(columns={"terr. code":"territory_code"},inplace= True)

/var/folders/f7/p98720yn5kn5tvm33x4lx_89rys50w/T/ipykernel_11202/94170790.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cs_25['terr. code'].replace({'RANCHI-III':'171501', 'RANCHI-IV':'171502','DHANBAD': '172201', 'DUMKA':'172301' ,'JAMSHEDPUR':'171101' }, inplace=True)


In [45]:
territory_sum

,territory_code,jktil,apollo,mrf,ceat,oth.,total,jktil_share,apollo_share,mrf_share,ceat_share,oth._share
0,11111,38769481.93,21571395.89,5.085429e+07,4.910873e+07,2.144556e+07,1.817495e+08,21.331277,11.868754,27.980436,27.020016,11.799517
1,11112,52277227.82,73072080.40,2.249952e+08,1.059769e+08,2.196012e+07,4.782815e+08,10.930222,15.278049,47.042418,22.157847,4.591463
2,11211,15898490.19,17215494.30,4.775022e+07,2.034884e+07,1.709610e+07,1.183092e+08,13.438090,14.551278,40.360549,17.199719,14.450364
3,11212,24091523.90,12612377.07,7.911216e+07,2.736805e+07,2.606716e+07,1.692513e+08,14.234176,7.451865,46.742434,16.170070,15.401455
4,11221,26010315.35,38490522.49,1.071392e+08,1.260654e+07,9.889687e+06,1.941363e+08,13.397968,19.826549,55.187629,6.493656,5.094199
...,...,...,...,...,...,...,...,...,...,...,...,...
173,171101,1975175.91,289022.33,3.409905e+06,1.015954e+07,4.848426e+05,1.631848e+07,12.103919,1.771135,20.895969,62.257852,2.971125
174,171501,0.00,0.00,0.000000e+00,8.093511e+06,7.944204e+05,8.887931e+06,0.000000,0.000000,0.000000,91.061807,8.938193
175,171502,0.00,0.00,0.000000e+00,7.178574e+05,0.000000e+00,7.178574e+05,0.000000,0.000000,0.000000,100.000000,0.000000
176,172201,0.00,0.00,0.000000e+00,2.899013e+06,0.000000e+00,2.899013e+06,0.000000,0.000000,0.000000,100.000000,0.000000


### Creating Pivot of monthly data

In [46]:

# Ensure 'period' is a datetime for correct sorting
monthly_data['period'] = pd.to_datetime(monthly_data['period'], format='%Y-%m')

monthly_data = monthly_data.sort_values(['dealer_code', 'period'])


In [47]:
def assign_cm_labels(group):
    max_period = group['period'].max()
    min_period = group['period'].min()
    
    # Create a full month range
    full_range = pd.date_range(start=min_period, end=max_period, freq='MS')
    
    # Map actual months to labels
    label_map = {}
    for i, month in enumerate(sorted(full_range, reverse=True), start=1):
        label_map[month] = f'cm-{i}'
    
    # Assign labels only if period exists in label_map
    group['cm_label'] = group['period'].map(label_map)
    return group

monthly_data = monthly_data.groupby('dealer_code', group_keys=False).apply(assign_cm_labels)


/var/folders/f7/p98720yn5kn5tvm33x4lx_89rys50w/T/ipykernel_11202/2769496585.py:17: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  monthly_data = monthly_data.groupby('dealer_code', group_keys=False).apply(assign_cm_labels)


In [48]:
monthly_data = monthly_data[monthly_data['cm_label'] != "cm-13"]

In [49]:
monthly_data.columns

Index(['dealer_code', 'period', 'total_sales', 'total_invoices', 'total_units',
       'avg_invoices', 'avg_sales', 'avg_units_purchased', 'territory_code',
       'dealer_club_category', 'avg_sales_of_dealers_in_same_territory',
       'avg_orders_of_dealers_in_same_territory',
       'avg_units_of_dealers_in_same_territory',
       'avg_sales_of_dealers_in_same_dealer_club_category',
       'avg_orders_of_dealers_in_dealer_club_category',
       'avg_units_of_dealers_in_dealer_club_category', 'sas_amount',
       'rotation', 'terrwise_rotation', 'dealerclub_wise_rotation', 'exposure',
       'collection', 'outstanding', 'outstanding_0_30', 'outstanding_30_45',
       'outstanding_45_60', 'outstanding_60+', 'total_credit_note_value',
       'avg_credit_note_value', 'offline', 'online', 'total_orders',
       'unique_claims', 'settled_claims', 'active_claims', 'visit_count',
       'cm_label'],
      dtype='object')

In [50]:
monthly_data_pivot = monthly_data.pivot(index='dealer_code', columns='cm_label', 
                   values=['total_sales', 'total_invoices', 'total_units',
       'avg_invoices', 'avg_sales', 'avg_units_purchased',
       'avg_sales_of_dealers_in_same_territory',
       'avg_orders_of_dealers_in_same_territory',
       'avg_units_of_dealers_in_same_territory',
       'avg_sales_of_dealers_in_same_dealer_club_category',
       'avg_orders_of_dealers_in_dealer_club_category',
       'avg_units_of_dealers_in_dealer_club_category', 'sas_amount',
       'rotation', 'terrwise_rotation', 'dealerclub_wise_rotation', 'exposure',
       'collection', 'outstanding', 'outstanding_0_30', 'outstanding_30_45',
       'outstanding_45_60', 'outstanding_60+', 'total_credit_note_value',
       'avg_credit_note_value', 'offline', 'online', 'total_orders',
       'unique_claims', 'settled_claims', 'active_claims', 'visit_count']).reset_index()

In [51]:
monthly_data_pivot.columns = [
    '_'.join([str(c) for c in col if c != '']).strip()  
    if isinstance(col, tuple) else str(col)  
    for col in monthly_data_pivot.columns
]


In [52]:
monthly_data_pivot.columns

Index(['dealer_code', 'total_sales_cm-1', 'total_sales_cm-10',
       'total_sales_cm-11', 'total_sales_cm-12', 'total_sales_cm-2',
       'total_sales_cm-3', 'total_sales_cm-4', 'total_sales_cm-5',
       'total_sales_cm-6',
       ...
       'visit_count_cm-11', 'visit_count_cm-12', 'visit_count_cm-2',
       'visit_count_cm-3', 'visit_count_cm-4', 'visit_count_cm-5',
       'visit_count_cm-6', 'visit_count_cm-7', 'visit_count_cm-8',
       'visit_count_cm-9'],
      dtype='object', length=385)

In [53]:
##Days between Purchases, Movement Counts

d1 = pd.merge(monthly_data_pivot, days_between_purchase, on='dealer_code', how='left') \
.merge(movement_counts, on='dealer_code', how='left') 

In [54]:
customer_master.head()

,dealer_code,dealer_class,dealer_group,latitude,longitude,dealer_pincode,address,territory_code,city,dealer_name,...,dealer_club_category,current_sas,exposure,total_outstanding,outstanding_0_30,outstanding_30_45,outstanding_45_60,outstanding_60+,collection_mtd,dealership_age
0,1116547,PD,Z001,25.861327700000,85.774513800000,700114,ANIMA MOTORS,111102,KOLKATA,ANIMA MOTORS,...,BLUE CLUB,329730,0,261490,120659,5689,115297,19845,40000,6
1,1120419,CO,Z001,24.018627,84.101672,822101,AJIT TRADING,171502,DALTONGANJ,AJIT TRADING,...,BLUE CLUB,605449,0,146564,168651,0,0,-22087,27568,3
2,1104410,TP,Z001,22.820127100000,70.834233100000,363641,SHAH KESHAVLAL MANJIBHAI & CO.,752101,MORBI,SHAH KESHAVLAL MANJIBHAI & CO.,...,PLATINUM CLUB,2500001,0,390448,401637,0,-11189,0,1116936,17
3,1116778,SW,Z001,26.424437,80.314853,208006,GAYATRI TRADERS,371102,KANPUR,GAYATRI TRADERS,...,DIAMOND CLUB,5500000,973306,6473306,6473306,0,0,0,1688512,6
4,1100011,PD,Z001,None,None,132001,B.R. AUTO STORE,321203,KARNAL,B.R. AUTO STORE,...,PLATINUM CLUB,2555421,79923,2635344,2296141,339203,0,0,351837,18


In [55]:
customer_master.columns

Index(['dealer_code', 'dealer_class', 'dealer_group', 'latitude', 'longitude',
       'dealer_pincode', 'address', 'territory_code', 'city', 'dealer_name',
       'creation_date', 'sabtaluka_name', 'mobile_number', 'mobile_number2',
       'gst_number', 'dealer_group_description', 'dealer_class_description',
       'dealer_state', 'ABC_class', 'active_last_120_days',
       'active_last_60_days', 'dealer_status', 'dealer_revival',
       'dealer_club_category', 'current_sas', 'exposure', 'total_outstanding',
       'outstanding_0_30', 'outstanding_30_45', 'outstanding_45_60',
       'outstanding_60+', 'collection_mtd', 'dealership_age'],
      dtype='object')

In [56]:
d1 = pd.merge(monthly_data_pivot, customer_master[['dealer_code', 'territory_code','dealer_club_category','dealership_age']], on='dealer_code', how='left') \
.merge(days_between_purchase, on='dealer_code', how='left') \
.merge(movement_counts, on='dealer_code', how='left') \
.merge(territory_sum, on='territory_code', how='left')

In [57]:
7379/d1.shape[0]

0.9634417025721373

In [58]:
d1.isna().sum()

dealer_code             0
total_sales_cm-1        0
total_sales_cm-10    1274
total_sales_cm-11    1330
total_sales_cm-12    1477
                     ... 
jktil_share          7473
apollo_share         7473
mrf_share            7473
ceat_share           7473
oth._share           7473
Length: 403, dtype: int64

In [59]:
d1 = pd.merge(d1, last_billed[['dealer_code','last_billed_days','churn_status']],on='dealer_code', how='left')

In [61]:
territory_master.head()

,territory_code,territory_name,plant_depot_code,area_code,region_code,employee_number,territory_type,email_id,territory_category,region_name,territory_scope,zone,so_name,so_designation,active_status_territory,dealer_count,mobile_number
0,110001,Nepal,1102,1101,1682,00000000,SO,NA,COM,Nepal,S,Nepal,None,None,A,45,None
1,110002,Bhutan,1102,1101,1682,00000000,SO,NA,COM,Nepal,S,Nepal,None,None,A,57,None
2,111101,KOLKATTA NORTH-I,1111,1181,1181,00020951,SO,tmkolkattanorth-i@jkmail.com,COM,KOLKATA,S,EAST,Chaman Preet Singh,Sr.Sales Officer,A,195,None
3,111102,KOLKATTA NORTH-II,1111,1181,1181,00024871,SO,tmkolkattanorth-ii@jkmail.com,COM,KOLKATA,S,EAST,Amit Kumar Singh,Sales Officer,A,315,None
4,112101,KOLKATTA SOUTH-I,1121,1181,1181,00025191,SO,tmkolkattasouth-i@jkmail.com,COM,KOLKATA,S,EAST,Soumya Roudra Das,Sr.Sales Officer,A,293,9674828204


In [65]:
d1 = pd.merge(d1,territory_master[['zone', 'territory_code', 'region_name','dealer_count']],on='territory_code', how='left')

In [66]:
d1.to_csv('offset_features.csv', index=False)